# EEP Combine Content and Market Sheets, Then Append Student Names to Analytics Report (to be reused)

Document Instructions: https://docs.google.com/document/d/1Hh09EyCa4iRDEybzRZOcHJr5-azNS5GOTAM1Wl1ymPQ/edit?usp=sharing

Video Tutorial: https://loom.com/share/c32602e3c17b480b922e957cd32b7c12

## Basic Jupyter Notebook Operations

to run a cell (a gray box): CTRL+ENTER or SHIFT+ENTER

to comment/ uncomment a line or selected lines: CTRL+/ 

In [ ]:
# pandas is a Python library for working with data tables. It contains helpful functions that we can call on.
import pandas as pd
import numpy as np

## 1. Combine Content Sheet and Market Sheet
Content Sheet link: https://docs.google.com/spreadsheets/d/1lyt-nnXbloOmpOLV6qVQPvcVKFzvgIKOZeX2G5vR2B0/edit#gid=1548859272

Market Sheet link: https://docs.google.com/spreadsheets/d/1NNPYU5KycjL2gFdJYh0ogwjWQjfcOFWV/edit#gid=409638312

### 1.1 Read In the Data Tables
Download a specific sheet from Google Sheet by clicking File -> Download -> Comma-separated values (.csv, current sheet).

Get the CSV file's directory path by right-clicking on the CSV file, click "Properties", then copy paste the text to the right of "Location". Change all the backslashes ("\\") to forwardslashes ("/"), then add the full file name (e.g. file_name.csv) at the end.

In [ ]:
today_date = "2021-08-20" # change to today's date in YYYY-MM-DD format

folder_directory_path = "C:/Users/tsaie/OneDrive/Desktop/EEP/DBM/Appending to Analytics Report" + "/"

content_sheet_file_name = ".csv"
market_sheet_file_name = ".csv"
market_sheet_last_row_index = 154 # we will remove all the unwanted rows below this row

'''
To easily convert student names to their corresponding team numbers,
create a file of just 2 columns: "Student" (student full name) and "Team Number" (just the number).
Create this file by downloading and cleaning the first tab in the Time and Accomplishment Tracker.
'''
student_team_num_file_name = "students and teams.csv"


content_sheet_directory_path = folder_directory_path + content_sheet_file_name
print(content_sheet_directory_path)
market_sheet_directory_path = folder_directory_path + market_sheet_file_name
print(market_sheet_directory_path)
student_team_num_directory_path = folder_directory_path + student_team_num_file_name
print(student_team_num_directory_path)

### (For the rest of Section 1.1 and 1.2, simply run the cells. No need to change anything in the cells unless column names have changed.)

In [ ]:
content_sheet = pd.read_csv(content_sheet_directory_path)
content_sheet_primary_key = "Final URL (ONLY IF LIVE)"

# Make sure the primary key is in string/text format:
content_sheet[content_sheet_primary_key] = content_sheet[content_sheet_primary_key].astype("str")

# Display table
content_sheet

In [ ]:
market_sheet = pd.read_csv(market_sheet_directory_path).iloc[:market_sheet_last_row_index+3] # remove unecessary rows
market_sheet_primary_key = "market url"

# Make sure the primary key is in string/text format:
market_sheet[market_sheet_primary_key] = market_sheet[market_sheet_primary_key].astype("str")

# Display table
market_sheet

In [ ]:
student_team_num = pd.read_csv(student_team_num_directory_path)
student_team_num_primary_key = "Name"

# make sure the primary key is in string/text format:
student_team_num[student_team_num_primary_key] = student_team_num[student_team_num_primary_key].astype("str")
student_team_num["Team Number"] = student_team_num["Team Number"].apply(lambda x: str(x).replace(".0",""))

# set index to student name
student_team_num = student_team_num.set_index(student_team_num_primary_key)

# Display table
student_team_num

### 1.2 Create and Fill a New DataFrame 

In [ ]:
is_successful = True

# Define column names
primary_key = 'URL'
URL_type = 'URL Type'
students_involved = 'Students Involved'
team_number = 'Team Number'

# Create DataFrame using the column names we just defined
combined = pd.DataFrame(columns = [primary_key, URL_type, students_involved, team_number])

# Append from Content Sheet to the new DataFrame row by row
for index, row in content_sheet.iterrows():
    try:
        url = row[content_sheet_primary_key]
        content_type = str(row["Content Type"])
        students = str(row["Students who've worked on article"]) if str(row["Students who've worked on article"]) != "nan" else None

        # Checks if url exists for this row
        if url != "nan": 
            
            teams = ""
            # if students is not empty, find all the teams involved based on the student names and the student_team_num table
            if students != None:
                for name in students.split(", "):
                    if " (coach)" in name:
                        name = name[:-len(" (coach)")]
                    if name in student_team_num.index:
                        team_num = str(student_team_num.loc[name].get("Team Number"))
                        if team_num not in teams:
                            teams += "Team " + team_num + ", "
                if len(teams) > 0:
                    teams = teams[:-2] # to remove the final ", "

            # Append the row
            combined = combined.append({primary_key : url, 
                                        URL_type : content_type, 
                                        students_involved : students, 
                                        team_number : teams},
                                        ignore_index = True)
    except:
        print("Content Sheet | Index ", index, "produced error |", row[content_sheet_primary_key])
        is_successful = False

        
# Append from Market Sheet to the new DataFrame row by row
for index, row in market_sheet.iterrows():
    try:
        url = row[market_sheet_primary_key]
        students = str(row["Students who've worked on market"]) if str(row["Students who've worked on market"]) != "nan" else None
        team_num = str(row["Team Number"]) if str(row["Team Number"]) != "nan" else None

        # Checks if url exists for this row
        if url != "nan": 

            # Append the row
            combined = combined.append({primary_key : url, 
                                        URL_type : "Market", 
                                        students_involved : students, 
                                        team_number : team_num},
                                        ignore_index = True)
    except:
        print("Market Sheet | Index", index, "produced error") # errors shouldn't happen
        is_successful = False

if is_successful:
    print("Successfully combined Content sheet and Market sheet. Please move on to the next cell.")

### 1.3 Save the Result to Your Local PC, in the Same Folder as the Source Files

If you wish to save the result in another folder, replace folder_directory_path with the directory path you want.

Name this file “RESULT YYYY-MM-DD Student Names for Content and Market URLs YYYY-MM-DD v1.xlsx”
- Change v1 to the version number for today, starting at v1, then v2, v3, etc.


In [ ]:
result_file_name = "RESULT {date} Student Names for Content and Market URLs v1.xlsx".format(date=today_date)
combined.to_excel(folder_directory_path + result_file_name)
print("Successfully saved resulting file as", result_file_name)

## 2. Append Student Names and Team Numbers to Analytics Report

### 2.1 Read In the Data Tables

Note: It is ok to have a CSV file open when you read it in, but to successfully read in an Excel file (.xlsx), you cannot have the file open when you run the cell, or it will produce an error. You can open the Excel file after running the cell and reading in the file.

In [ ]:
append_from = pd.read_excel(folder_directory_path + result_file_name)
append_from_primary_key = 'URL'

# Make sure the primary key is in string/text format, and remove any slash at the end of the URL
append_from[append_from_primary_key] = append_from[append_from_primary_key].apply(lambda x: str(x)[:-1] if str(x)[-1] == "/" else str(x))

# Set 'URL' as index
append_from = append_from.set_index(append_from_primary_key)

# Display table
append_from

In [ ]:
# append_to is the dataframe that we want to append data to. In this case the append_to is the analytics report.
analytics_report_file_name = ".csv"
append_to = pd.read_csv(folder_directory_path + analytics_report_file_name)
append_to_primary_key = 'URL'

# make sure the primary key is in string/text format:
append_to[append_to_primary_key] = append_to[append_to_primary_key].astype("str")

# Display table
append_to

### 2.2 Append Student Names and Team Numbers to Analytics Report

Just run the cells, no need to change anything except for the name of the resulting file.

In [ ]:
append_to_col_1 = "URL Type (appended {date})".format(date=today_date)
append_to_col_2 = "Students Involved (appended {date})".format(date=today_date)
append_to_col_3 = "Teams Involved (appended {date})".format(date=today_date)
append_to[append_to_col_1] = pd.Series(dtype="str")
append_to[append_to_col_2] = pd.Series(dtype="str")
append_to[append_to_col_3] = pd.Series(dtype="str")

for index, row in append_to.iterrows():
    row_id = row[append_to_primary_key]
    if row_id in append_from.index:
        append_from_row = append_from.loc[row_id]
        append_to.at[index, append_to_col_1] = str(append_from_row.get(URL_type))
        append_to.at[index, append_to_col_2] = str(append_from_row.get(students_involved))
        append_to.at[index, append_to_col_3] = str(append_from_row.get(team_number)) if str(append_from_row.get(team_number)) != "nan" else None
        print("Index: {i} | {var1}: {var1_result} | {var2}: {var2_result} | {var3}: {var3_result}".format(
               i=index, 
               var1=URL_type, var1_result=append_from_row.get(URL_type),
               var2=students_involved, var2_result=append_from_row.get(students_involved),
               var3=team_number, var3_result=append_from_row.get(team_number)))
    else:
        print("couldn't find " + row_id) # don't worry, it is normal that some URLs can't be found

In [ ]:
# Display final result
append_to

### 2.3 Save the Result to Your Local PC, in the Same Folder as the Source Files
If you wish to save the result in another folder, replace folder_directory_path with the directory path you want.

Name this file “RESULT YYYY-MM-DD Analytics Report Appended Student Names and Teams v1.xlsx”
- Change v1 to the version number for today, starting at v1, then v2, v3, etc.

In [ ]:
result_file_name = "RESULT {date} Analytics Report Appended Student Names and Teams v1.xlsx".format(date=today_date)
append_to.to_excel(folder_directory_path + result_file_name)
print("Successfully saved resulting file as", result_file_name)

## End. Remember to spot check the file for errors.